In [1]:
import os
import sys
import numpy as np
import torch
import torchvision.models as models
from torch.autograd import Variable
from common.params_inf import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [5]:
# Download ResNet
resnet50 = models.resnet50(pretrained=True)
# Chop-off last FC layer
print(list(resnet50.children())[-1])
chopped_resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])
# CUDA
chopped_resnet50.cuda()
# Last layer is now avgpool2d
print(list(chopped_resnet50.children())[-1])

Linear(in_features=2048, out_features=1000, bias=True)
AvgPool2d(kernel_size=7, stride=7, padding=0, ceil_mode=False, count_include_pad=True)


In [6]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    classifier.eval()
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb_X(data, batchsize):
        pred = classifier(Variable(torch.FloatTensor(dta).cuda()))
        out[idx*batchsize:(idx+1)*batchsize] = pred.data.cpu().numpy().squeeze()
    return out

In [7]:
cold_start = predict_fn(chopped_resnet50, fake_input_data_cf, BATCH_SIZE)

In [8]:
%%time
features = predict_fn(chopped_resnet50, fake_input_data_cf, BATCH_SIZE)

CPU times: user 2.05 s, sys: 397 ms, total: 2.44 s
Wall time: 2.44 s


In [9]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/2.44))

Images per second 524.5901639344262
